### Import 

In [ ]:
import numpy as np
from scipy import stats

import cvxopt as opt
from cvxopt import *
from cvxopt import blas, solvers

import cvxpy as cp

import pandas_datareader.data as web
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import pandas as pd

# !pip install PyPortfolioOpt <- ★실행 시 설치가 필요합니다.
import pypfopt
from pypfopt.efficient_frontier import EfficientFrontier

import scipy.optimize as sco

from plotly.subplots import make_subplots
from sklearn import linear_model

# 1. Data

In [ ]:
class Data:
    '''
    Description: 투자 분석에 사용할 자산 구성
     (Constructor: param)
      - name: 자산 이름 설정
      - isin_ind: 산업군 데이터 포함 여부
      - isin_add: 추가 데이터 포함 여부
      - ind_num: 산업군 데이터 종류
      - add_lst: 추가 데이터 선택 목록
    
    Method
     (Public)
      - prepare_data: 투자시작/종료 날짜 및 룩백기간을 고려한 최종 자산 구성
      - check_date: 투자시작/종료 날짜 및 룩백기간이 선택한 자산과 최소 투자가능날짜(1970-02)에 적합한지 확인
      - make_dataset: 선택한 데이터를 이용해 기간 고려하지 않고 자산 구성
     (Private)
      - _error_check: 에러 체크
      - _date_str_to_int: string형태의 날짜를 int로 변경
      - _cal_date_with_month: (특정 날짜) - (N개월) 계산
      - _update_min_date: 선택한 자산의 사용가능 기간을 고려해 최소 투자가능날짜 변경
      - _get_fama_french: fama-french 3 factor 데이터를 file로부터 읽어와 가공
    '''
    
    # Data class Constructor
    def __init__(self, name, isin_ind, isin_add=False, ind_num = 10, add_lst = []):
        self.name = name
        self.isin_ind = isin_ind
        self.isin_add = isin_add
        self.ind_num = ind_num
        self.add_lst = add_lst
        self.fin_df = pd.DataFrame([])
        self.bm_df = pd.DataFrame([])
        self.ff3_df = pd.DataFrame([])
        
        self.origin_df = pd.DataFrame([])
        self.origin_mb = pd.DataFrame([])
        self.origin_ff3 = pd.DataFrame([])
        
        self.add_asset_dict = {0:"US BOND 7-10", 1: "US BOND 20+", 2: "GOLD",
                              3: "COMMODITY" , 4:"S&P ETF"}
        self.limit_date_dict = {0:"2002-08", 1: "2002-08", 2: "2005-02",
                              3: "2006-03" , 4:"1993-02"}
        self.min_date = '1970-02'
        self.max_date = '2020-09'
        
        if not self._error_check():
            return 0
        
    # error 확인
    def _error_check(self):
        if not self.isin_ind and not self.isin_add:
            print("[ERROR] 분석에 사용할 자산을 입력해주세요.")
            return False   
        if not isinstance(self.add_lst, list):
            print("[ERROR] 리스트 형태로 입력해주세요.")
            return False
        return True
    
    # string 타입의 date를 int형으로 바꿔주는 함수
    def _date_str_to_int(self, date):
        y, m = date.split('-')
        y, m = int(y), int(m)
        return y,m

    # string date로부터 mon개월 이전의 날짜 계산
    def _cal_date_with_month(self, date, mon):
        y, m = self._date_str_to_int(date)
        
        y -= mon // 12
        mon = mon % 12
        
        if m-mon < 1:
            y -= 1
            m = m + 12 - mon
        else:
            m -= mon

        return y,m

    # 선택한 자산을 토대로 최소 투자가능 날짜 계산
    def _update_min_date(self):
        if self.isin_add:
            for i in self.add_lst:
                new_date = self.limit_date_dict[i]
                
                y,m = self._date_str_to_int(self.min_date)
                y2, m2 = self._date_str_to_int(new_date)
                
                if y < y2:
                    self.min_date = new_date
                elif y == y2 and m < m2:
                    self.min_date = new_date
                    
    # 투자 시작/종료, 룩백 날짜를 고려한 최종 자산 가공  
    def prepare_data(self, inv_start,inv_end, lookback_period):
        self.make_dataset()
        
        y,m = self._cal_date_with_month(inv_start,lookback_period)
        look_start = '{0:04d}-{1:02d}'.format(y,m)
        
        self.fin_df = self.origin_df.loc[look_start:inv_end]
        self.bm_df = self.origin_bm.loc[look_start:inv_end]
        self.ff3_df = self.origin_ff3.loc[look_start:inv_end]
            
        
    # 투자시작/종료 날짜 및 룩백기간이 선택한 자산과 최소 투자가능날짜(1970-02)에 적합한지 확인
    def check_date(self, inv_start, inv_end, lookback_period):
        self._update_min_date()
        
        # inv_start와 룩백확인
        miny,minm = self._date_str_to_int(self.min_date)
        ly,lm = self._cal_date_with_month(inv_start,lookback_period)
        
        if ly < miny:
            print("[ERROR] 투자시작날짜와 lookback 기간이 제한된 시작날짜({})를 벗어납니다. 다시 설정해주세요".format(self.min_date))
            return False
        elif ly == miny and lm < minm:
            print("[ERROR] 투자시작날짜와 lookback 기간이 제한된 시작날짜({})를 벗어납니다. 다시 설정해주세요".format(self.min_date))
            return False
        
        # inv_end확인
        maxy,maxm = self._date_str_to_int(self.max_date)
        y2,m2 = self._date_str_to_int(inv_end)
        
        if y2 > maxy:
            print("[ERROR] 투자종료날짜가 제한된 종료날짜({})를 벗어납니다. 다시 설정해주세요".format(self.max_date))
            return False
        elif y2 == maxy and m2 > maxm:
            print("[ERROR] 투자종료날짜가 제한된 종료날짜({})를 벗어납니다. 다시 설정해주세요".format(self.max_date))
            return False
        
        return True
    
    # fama-french 3 factor 데이터를 file로부터 읽어와 가공
    def _get_fama_french(self):
        # Description : file 을 통해 data를 로드하고 필요 없는 행 제거
        # input : none
        # output : factor 수익률 / type: dataframe


        # file을 통해 data load
        ff_factors = pd.read_csv('data/F-F_Research_Data_Factors.CSV', skiprows = 3, nrows = 1131, index_col = 0)

        # Format the date index
        ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')

        # Convert from percent to decimal
        ff_factors = ff_factors.apply(lambda x: x/ 100)
        return ff_factors

    # 선택한 데이터를 이용해 기간 고려하지 않고 자산 구성
    def make_dataset(self):
        total_asset_df = pd.DataFrame([])

        # US industry data
        if self.isin_ind:
            ind= web.DataReader('{}_Industry_Portfolios'.format(self.ind_num), 'famafrench', start=self.min_date, end='2020-09')
            ind_ret=ind[0]/100
            total_asset_df = pd.concat([total_asset_df, ind_ret], axis=1)

        # Add data
        if self.isin_add:
            
            # 10개 산업군 주식데이터는 위에서 다루었으니 주식관련 데이터를 제외하고
            # 다양한 투자 종목을 추가해 Risk parity, MVO의 전체적인 분산효과를 높혀보기 위하여
            # 여러 투자 자산을 추가 해봅니다.
            # ETF 운용사 선택 기준은 과거 데이터가 길게 남아 있는 것으로 우선하였습니다.

            # iShares 7-10 Year Treasury Bond ETF (IEF)       - 미국중기채       (2002.08 ~ )
            # iShares 20+ Year Treasury Bond ETF (TLT)        - 미국장기채       (2002.08 ~ )
            # iShares Gold Trust (IAU)                        - 금               (2005.02 ~ )
            # Invesco DB Commodity Index Tracking Fund (DBC)  - 원자재           (2006.03 ~ )
            # SPDR S&P 500 ETF Trust (SPY)                    - S&P 지수추종ETF  (1993.02 ~ )

            assets = web.get_data_yahoo(['IEF','TLT','IAU','DBC','SPY'], start='1989-12-01', end='2020-09-01',interval='m')['Adj Close'].pct_change()

            assets.rename(columns={'IEF':'US BOND 7-10'},inplace=True)
            assets.rename(columns={'TLT':'US BOND 20+'},inplace=True)
            assets.rename(columns={'IAU':'GOLD'},inplace=True)
            assets.rename(columns={'DBC':'COMMODITY'},inplace=True)
            assets.rename(columns={'SPY':'S&P ETF'},inplace=True)

            assets.index = assets.index.strftime("%Y-%m") 
            assets.index = pd.to_datetime(assets.index).to_period("M")
            
            new_asset = assets.iloc[:, self.add_lst]
            
            total_asset_df = pd.concat([total_asset_df, new_asset],axis=1)
            
        self.origin_df = total_asset_df.copy()
        
        # benchmark
        sp500 = web.get_data_yahoo('^GSPC', start='1970-02-01', end='2020-09-01',interval='m')
        sp500c = sp500['Adj Close'].pct_change().dropna()
        sp500c.index = sp500c.index.strftime("%Y-%m") 
        
        self.origin_bm = sp500c.copy()
        
        # Fama-French 3 Factor
        ff_data = self._get_fama_french()
        ff_data.index = ff_data.index.strftime("%Y-%m")
        ff_data = ff_data[self.min_date:'2020-09']
        
        self.origin_ff3 = ff_data.copy()

# 2. Portfolio Optimization

In [ ]:
class Portfolio_Optimization:
    '''
    Description: 포트폴리오 최적화 모델
     (Constructor: param)
      - est_lst: lookback 기간 동안의 estimate
          est_lst[0]: mean lst ex) [0.5,0.3, ..., 0.7] 
          est_lst[1]: var lst ex) [...]
          est_lst[2]: cov matrix ex) [[...],[...],...]
          est_lst[3]: holding-period return 
      - ret: lookback 기간 바로 이후 자산 구성
     (Constructor: default param)
      - step_size: Lasso_n에 사용되는 step_size 목록
    
    Method
     (Public)
      - Equal_weight: 동일 가중치 모델
      - Mean_variance: 포티플리오의 risk를 최소화하는 평균-분산 모델
      - Risk_parity: 각 자산마다 risk를 동등하게 갖는 Risk parity 모델
      - Ridge: Ridge규제 추가 모델
      - Lasso: Lasso규제 추가 모델
      - Lasso_n: Lasso규제를 이용해 n개의 자산만으로 포트폴리오 구성하는 최적의 람다를 이용해 가중치를 계산하는 모델
      - Factor1: Mkt Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
      - Factor2: SMB Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
      - Factor3: HML Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
      - AllSeason: All season 포트폴리오
      - GB:  Golden butterfly 포트폴리오
      
      - regression: Factor와 자산을 이용한 회귀분석 진행
      
     (Private)
      - _L1_norm: Lasso 규제 이용함수
     
    '''
    
    # Portfolio_Optimization constructor
    def __init__(self,est_lst,ret):
        self.est_lst = est_lst
        self.ret = ret
        self.step_size = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]  

    # EQW model 
    # 목적 : 1/n 씩 나눠주기.
    # 사용 최적화 library : -
    def Equal_weight(self):
        # 각 자산의 가중치를 1/n로 설정합니다.
        weight_dict = {}
        for i in self.est_lst[0].index:
            weight_dict.update({i:1/len(self.ret)})
        weight_lst = pd.Series(weight_dict)
        return weight_lst
    
    # M-V model (w/o short selling)
    # 목적 : 포트폴리오 risk를 최소화하는 포트폴리오의 가중치 벡터 weight_lst 구하기
    # 사용 최적화 library : scipy.optimize
    def Mean_variance(self):
      
        n=len(self.est_lst[0])

        # 목적함수 값을 반환하는 함수.
        # np.matrix(self.est_lst[2]) 가 시그마(공분산테이블)
        def minvar(wt):
            res_minvar = np.array(np.sqrt(wt.T @ np.matrix(self.est_lst[2]) @ wt)).flatten()
            return res_minvar

        # 초기비중설정 1/n
        w = np.array([1/n]*n) 
        
        # 비중의 합이 1이 되도록함.
        # 'eq' 를 썻을때 기본적으로 0과 같은지 비교해서 np.sum(x) == 1 이 아니라
        #  np.sum(x) - 1 == 0 의 개념으로 비교한다고 보면 됩니다.
        cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})
        
        # 각 자산의 선택 가능 범위 0 ~ 1 (공매도 x)
        bnds = ((0,1),) *n

        
        # 목적함수 minvar가 최소가 되도록 scipy.optimize.minimize 사용
        # 출처 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html - scipy reference 공식
        opt = sco.minimize(minvar, w, method='SLSQP', bounds=bnds, constraints=cons)
        
        # 이후 나온 최적값을 pandas.Series 로 반환.
        weight_dict = {}
        idx=0
        for col in self.est_lst[0].index:
            weight_dict.update({col:opt['x'][idx]})
            idx+=1
        weight_lst = pd.Series(weight_dict)
        return weight_lst

    ###Q3. Risk parity를 어떻게 구현했는지 궁금한데 이는 코드를 제출하면 확인해보겠습니다.
    # Risk Parity model
    # 목적 : 각 자산별 risk를 동등하게 가져가는 비율의 weight_lst 구하기
    # 사용 최적화 library : scipy.optimize# EQW model
    def Risk_parity(self):
        n=len(self.est_lst[0])
        

        # 자산의 위험기여비율과 1/n 과의 차이제곱합을 return
        # 출처 : https://steemit.com/dclick/@thrufore/day3-risk-parity-with-python-1539700241095
        def RiskParity(wt):
            # 비중에따른 포트폴리오 위험을 wt @ est_lst[2] @ wt.T 순서로 곱해 n*n 행렬로 한다. (est_lst[2] => 공분산행렬)
            VarianceMatrix = self.est_lst[2].multiply(wt, axis = 1).multiply(wt, axis = 0) 
            
            
            # 자산의 위험기여도는 행렬로 표현한 포트폴리오 위험의 각 행의 합과 같음.
            # *(https://steemit.com/kr/@thrufore/high-risk-high-return-yes -자산의 위험 기여도)
            # 이를 전체 포트폴리오의 risk로 나눴을때 자산의 위험기여비율을 구할수 있음 (RCratio => (각행합 / 포트폴리오risk))
            RCratio = VarianceMatrix.sum() / VarianceMatrix.sum().sum()


            # 자산의 위험기여비율이 모두 1/n이 되도록 하는것이 좋은데
            # 1/n과 각 자산의 위험기여비율의 차이제곱을 구한다. => (1/n - RCratio[각자산])^2
            # 이후 그합을 구하는데 이값이 낮을수록 각 위험비율이 1/n으로 수렴하고 있음을 확인 할 수있다.
            return np.sum(((1.0/n)-np.array(RCratio))**2)
        
        # 초기비중설정 1/n
        w = np.array([1/n]*n) 

        # 비중의 합이 1이 되도록함.
        # 'eq' 를 썻을때 기본적으로 0과 같은지 비교해서 np.sum(x) == 1 이 아니라
        #  np.sum(x) - 1 == 0 의 개념으로 비교한다고 보면 됩니다.
        cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})

        # 각 자산의 선택 가능 범위 0 ~ 1
        bnds = ((0,1),) *n

        # optimizer 의 성능 향상을 위한 반복횟수 증가.. (너무 오래걸려서 사용안했습니다.)
        #option = {'maxiter': 100, 'ftol': 1e-60}     # ftol은 거의 0에 가까운 목표값, maxiter는 최대반복횟수
        #opt = sco.minimize(RiskParity, w, method='SLSQP', bounds=bnds, constraints=cons,options=option)

        
        # 목적함수 RiskParity가 최소가 되도록 scipy.optimize.minimize 사용
        # 출처 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html - scipy reference 공식
        opt = sco.minimize(RiskParity, w, method='SLSQP', bounds=bnds, constraints=cons)

        # opt.x 를 이용하여 risk가 적절히 나뉘었는지 확인
        #VarianceMatrix = est_lst[2].multiply(opt.x, axis = 1).multiply(opt.x, axis = 0)     
        #RCratio = VarianceMatrix.sum() / VarianceMatrix.sum().sum()
        #print("----risk----")
        #print(RCratio)

        # 이후 나온 최적값을 pandas.Series 로 반환.
        weight_dict = {}
        idx=0
        for col in self.est_lst[0].index:
            weight_dict.update({col:opt['x'][idx]})
            idx+=1
        weight_lst = pd.Series(weight_dict)
        return weight_lst

    # Ridge model
    # 사용 library : PyPortfolioOpt
    # 출처: https://pyportfolioopt.readthedocs.io/en/latest/EfficientFrontier.html#more-on-l2-regularisation
    def Ridge(self, g=1):
        
        # 자산의 평균과 수익률과 공분산 행렬을 이용해 EfficientFrontier 구성
        ef = EfficientFrontier(self.est_lst[0], self.est_lst[2])
        
        # L2규제(Ridge)를 목적함수에 추가(라이브러리에서 제공하는 함수 이용)
        ef.add_objective(pypfopt.objective_functions.L2_reg, gamma=g)
        
        # volatility를 최소화하는 가중치 계산
        weight = ef.min_volatility()
        
        # 가중치를 Series형태로 변경
        weight_lst = pd.Series(ef.clean_weights())
        
        return weight_lst

    # Lasso model에 이용하는 L1 규제 함수
    def _L1_norm(self, w, k):
        return k * cp.norm(w, 1)
    
    # Lasso model
    # 사용 library : PyPortfolioOpt
    # 출처: https://pyportfolioopt.readthedocs.io/en/latest/EfficientFrontier.html#more-on-l2-regularisation
    def Lasso(self, l=1):
        
 
        # 자산의 평균과 수익률과 공분산 행렬을 이용해 EfficientFrontier 구성
        ef = EfficientFrontier(self.est_lst[0], self.est_lst[2])
        

        # L1규제(Lasso)를 목적함수에 추가
        ef.add_objective(self._L1_norm, k=l)
        
        # volatility를 최소화하는 가중치 계산
        weight = ef.min_volatility()
        
        # 가중치를 Series형태로 변경
        weight_lst = pd.Series(ef.clean_weights())
        
        return weight_lst

    # Lasso_n model
    # Lasso규제를 이용해 n개의 자산만으로 포트폴리오 구성하는 최적의 람다를 이용해 가중치를 계산하는 모델
    def Lasso_n(self, user_input, total_asset_num):
        
        # 에러 처리
        if user_input > total_asset_num:
            print("ERROR: 사용자 입력({})이 전체 자산의 개수({})를 초과하였습니다.".format(user_input, total_asset_num))
            return np.nan
        
        # 변수 초기화
        LARGE = total_asset_num + 1
        
        beg = 0
        limit = 100

        # 최종
        fin_asset_num = 0
        fin_asset_weight = []
        
        # 최소(가장 최적의 값을 계산이 끝날때까지 저장하며 조건 만족하면 계속 업데이트됨)
        mini_asset_num = 0
        mini_asset_weight = []
        mini_diff = LARGE
        
        target_lambda = 0
        
        # 동일 자산 개수를 갖는 lambda를 찾았는지 나타내주는 flag
        inter_fin_flag = False

        same_num_limit = 5
        
        # step_size를 줄여가며 자산개수 n과 동일(1순위)하거나 가장 유사한 개수(2순위)를 갖는 "최소" lambda를 찾는 것이 목적
        for i in range(len(self.step_size)):
            
            # limit 업데이트
            tmp_limit = limit

            num1 = 1
            num2 = 1
            count1 = 0
            count2 = 0
            
            # 구간 내에서 step_size만큼 늘려가며 최적의 lambda를 찾음
            for k in np.arange(beg, limit, self.step_size[i]):
                k = round(k,i+1)
                
                # 현재 람다(k)를 이용해 Lasso 모델로 가중치 계산
                tmp_weight = self.Lasso(k)
                
                # 계산한 가중치를 토대로 포트폴리오를 구성하는 자산개수 계산
                weight_num = total_asset_num-(tmp_weight==0).sum()

                # 현재 자산 개수와 목표 자산 개수 차이
                diff = weight_num - user_input

                # 현재 자산의 개수가 목표 자산 개수보다 큰 경우
                if diff > 0:
                    
                    # 목표 자산과의 차이가 지금까지 나온 최소 차이(mini_dff)보다 작고
                    # 동일 자산 개수를 갖는 lambda를 찾지 못한 경우
                    if mini_diff > diff and not inter_fin_flag:
                        
                        # 최적의 값과 tmp_limit 업데이트
                        mini_diff = diff
                        mini_asset_weight = tmp_weight
                        mini_asset_num = weight_num
                        tmp_limit = k
                        target_lambda = k
                    
                    # 목표 자산과의 차이가 지금까지 나온 최소 차이(mini_dff)와 동일하고
                    # 동일 자산 개수를 갖는 lambda를 찾지 못한 경우
                    elif mini_diff == diff and not inter_fin_flag:
                        # 종종 최소 차이 lambda보다 조금 더 크게 했을 때 동일한 자산 개수를 찾는 경우가 존재하여
                        # 이를 고려하기 위해 바로 종료하는게 아니라 same_num_limit만큼 더 찾을 수 있도록 조정
                        if num1 >= same_num_limit:
                            # 최소의 lambda를 찾는 것이 목적이므로
                            # 저장된 최소 lambda보다 현재 람다(k)가 더 작으면 업데이트
                            if target_lambda > k:
                                target_lambda = k
                                mini_diff = diff
                                mini_asset_weight = tmp_weight
                                mini_asset_num = weight_num
                                tmp_limit = k                   
                            break
                        num1 += 1
                        
                    # 그 외의 경우 최대 30번만 더 진행하고 그 이상이면 해당 step_size에서 탐색 종료
                    else:
                        count1 += 1
                        if count1 > 30:
                            break
                
                # 현재 자산 개수가 목표 자산 개수보다 작은 경우
                elif diff < 0:

                    # 목표 자산과의 차이(절댓값)가 지금까지 나온 최소 차이(mini_dff)보다 작고
                    # 동일 자산 개수를 갖는 lambda를 찾지 못한 경우
                    if mini_diff > abs(diff) and not inter_fin_flag:
                        
                        # 최적의 값과 tmp_limit 업데이트
                        mini_diff = abs(diff)
                        mini_asset_weight = tmp_weight
                        mini_asset_num = weight_num
                        target_lambda = k

                    # 목표 자산과의 차이(절댓값)가 지금까지 나온 최소 차이(mini_dff)와 동일하고
                    # 동일 자산 개수를 갖는 lambda를 찾지 못한 경우
                    elif mini_diff == abs(diff) and not inter_fin_flag:
                        
                        # 바로 종료하는지 않고 same_num_limit만큼 더 찾을 수 있도록 조정
                        if num2 >= same_num_limit:
                            if target_lambda > k:
                                target_lambda = k
                                mini_diff = abs(diff)
                                mini_asset_weight = tmp_weight
                                mini_asset_num = weight_num
                                tmp_limit = k
                            break
                        num2 += 1
                        
                    # 그 외의 경우 최대 30번만 더 진행하고 그 이상이면 해당 step_size에서 탐색 종료
                    else:
                        count2 += 1
                        if count2 > 30:
                            break
                        

                # 현재 자산의 개수와 목표 자산 개수가 같은 경우
                else:
                    tmp_limit = k
                    
                    # flag가 False인 경우
                    if not inter_fin_flag:
                        
                        # 동일 자산 개수를 갖는 lambda를 찾았으므로 True로 변경 후 값 업데이트
                        inter_fin_flag = True
                        fin_asset_num = weight_num
                        fin_asset_weight = tmp_weight
                        target_lambda = k
                        
                    # flag가 True인 경우
                    else:
                        # 최소 lambda값을 찾는 것이 목적이므로
                        # 만약 target_lambda보다 현재 람다(k)보다 작으면 업데이트
                        if target_lambda > k:
                            fin_asset_num = weight_num
                            fin_asset_weight = tmp_weight
                            target_lambda = k                    
                    break
        
            # beg 조정
            if target_lambda < tmp_limit and target_lambda - self.step_size[i] >= 0:
                beg = target_lambda - self.step_size[i]
            elif target_lambda >= tmp_limit and tmp_limit - self.step_size[i] >= 0:
                beg = tmp_limit - self.step_size[i]
                

            limit = tmp_limit
        
        # 화면 출력
        if inter_fin_flag:
            print("LASSO: 자산 {}개의 포트폴리오를 구성하기 위한 최적의 lambda는 {}입니다.\n".format(fin_asset_num,target_lambda))
            return fin_asset_weight
        else:
            print("※NOTICE: 사용자의 입력({})과 가장 유사한 {}개의 자산을 구성하였습니다.".format(user_input, mini_asset_num))
            print("LASSO:자산 {}개의 포트폴리오를 구성하기 위한 최적의 lambda는 {}입니다.\n".format(mini_asset_num, target_lambda))
            return mini_asset_weight

        
    # Description : regression을 통해 상위 (자산군 수)/3 에 속하는 자산군 list 얻기
    # input : lookback period 동안의 자산군 수익률 데이터와 factor 데이터 / type:dataframe
    # output : 상위 (자산군 수)/3 에 속하는 자산군들의 리스트 / type: list
    def regression(self, data, ff_data) :
        X = ff_data[['Mkt-RF','SMB', 'HML']]
        Y = data[data.columns]
        
        # Linear Regression 부분
        regr = linear_model.LinearRegression()
        regr.fit(X, Y)

        #print('Intercept: \n', regr.intercept_)
        #print('Coefficients: \n', regr.coef_)
        regr.coef_ = regr.coef_.T
        coef = regr.coef_.argsort(axis=1)
        #print(coef)

        top_ind = int(len(data.columns)/3)

        Mkt_RF_lst = []
        SMB_lst = []
        HML_lst = []
        for i in range(len(coef)) :
            for j in range(len(coef[i])-1,len(coef[i])-1-top_ind, -1) :
                if i == 0 :
                    Mkt_RF_lst.append(data.columns[coef[i][j]])
                elif i == 1 :
                    SMB_lst.append(data.columns[coef[i][j]])
                elif i == 2 :
                    HML_lst.append(data.columns[coef[i][j]])

        top_ind_lst = []
        top_ind_lst.append(Mkt_RF_lst)
        top_ind_lst.append(SMB_lst)
        top_ind_lst.append(HML_lst)

        return top_ind_lst

    # Description : Mkt Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
    # input : 자산군의 수익률 데이터와 팩터 데이터 / type: dataframe
    # output : optimize를 통해 갱신된 weight / type: Series
    def Factor1(self, ret, ff_data) :        

        top_ind_lst = self.regression(ret, ff_data)
        
        weight_dict = {}
        for i in self.est_lst[0].index:
            if i in top_ind_lst[0]:
                weight_dict.update({i:1/len(top_ind_lst[0])})
            else :
                weight_dict.update({i:0})
        weight_lst = pd.Series(weight_dict)
        return weight_lst
    
    # Description : SMB Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
    # input : 자산군의 수익률 데이터와 팩터 데이터 / type: dataframe
    # output : optimize를 통해 갱신된 weight / type: Series      
    def Factor2(self, ret, ff_data) :  

        top_ind_lst = self.regression(ret, ff_data)
        
        weight_dict = {}
        for i in self.est_lst[0].index:
            if i in top_ind_lst[1]:
                weight_dict.update({i:1/len(top_ind_lst[1])})
            else :
                weight_dict.update({i:0})
        weight_lst = pd.Series(weight_dict)
        return weight_lst
    
    # Description : HML Factor에 의해 영향 받는 상위 (자산군 수)/3 개의 자산군
    # input : 자산군의 수익률 데이터와 팩터 데이터 / type: dataframe
    # output : optimize를 통해 갱신된 weight / type: Series
    def Factor3(self, ret, ff_data) :
        top_ind_lst = self.regression(ret, ff_data)
        
        weight_dict = {}
        for i in self.est_lst[0].index:
            if i in top_ind_lst[2]:
                weight_dict.update({i:1/len(top_ind_lst[2])})
            else :
                weight_dict.update({i:0})
        weight_lst = pd.Series(weight_dict)
        return weight_lst

    
    ### Q1. All season과 Golden butterfly는 정해진 비율로 rebalance하는게 맞나요?
    # -> 예 정해진 비율로 rebalance하고 있습니다. 정해진 비율은 아래 코드와 같습니다.
    
    # All Season Portfolio weight
    def AllSeason(self) :
        weight_dict = {'US BOND 7-10':0.15,'US BOND 20+':0.4,'GOLD':0.075,'COMMODITY':0.075,'S&P ETF':0.3}
        weight_lst = pd.Series(weight_dict)
        return weight_lst
    
    # Golden Butterfly weight
    def GB(self) :
        weight_dict = {'US BOND 7-10':0.2,'US BOND 20+':0.2,'GOLD':0.2,'COMMODITY':0,'S&P ETF':0.4}
        weight_lst = pd.Series(weight_dict)
        return weight_lst
    
#     def _L1_L2_norm(self, w, k, gamma):
#         return k * cp.norm(w, 1) + gamma*cp.norm(w, 2)
    
#     def Elastic(self, l=1, g=1):
#         ef = EfficientFrontier(self.est_lst[0], self.est_lst[2])
#         ef.add_objective(self._L1_L2_norm, k=l, gamma= g)
#         weight = ef.min_volatility()
#         weight_lst = pd.Series(ef.clean_weights())
#         return weight_lst

# 3. BackTest

In [ ]:
class BackTest:
    '''
    Description: 백테스트 분석
     (Constructor: param)
      - data: 포트폴리오 구성 자산 수익률
      - data_name: 자산 이름
      - ff_data: 팩터 모델
      - reopt_period: re-optimize 주기
      - rebal_period: rebalancing 주기
      - lookback_period: lookback 기간
      - inv_start: 과거 투자시작 날짜
      - inv_end: 과거 투자종료 날짜
      - amount: 투자금액
     (Constructor: default param)
      - method_dict: 최적화 모델 종류
      - res_df: 백테스트 진행 과정에서 구한 포트폴리오 수익률(모델별)
      - weight_df_dict: 백테스트 진행 과정에서 구한 포트폴리오 가중치(모델별) 
    
    Method
     (Public)
      - start_backtest: 사용자로부터 입력된 포트폴리오 최적화 모델을 활용한 백테스트 진행
     (Private)
     - _date_str_to_int: date(YYYY-MM)형식을 year, mon 분리해서 int로 return
     - _cal_date_with_month: date(YYYY-MM형식)와 mon(개월 수) 연산
     - _df_to_estimate: 최적화 문제를 풀기 위해 필요한 estimate 계산
     - _pfo_return: 자산 평균 수익률과 가중치를 이용한 포트폴리오 return 계산
     - _rebalance_pfo: 포트폴리오 리밸런싱 진행
     
    '''
    
    # BackTest constructor
    def __init__(self, data, data_name, ff_data, reopt_period, rebal_period, lookback_period, inv_start, inv_end, amount=1):
        """
        === Description ===

        * parameter *
        data: historical data / type: dataframe
        data_name: name of assets / type: str
        reopt_period: re-optimizing assets weights period (month) / type: int
        rebal_period: rebalancing(evaluation) period (month) / type: int
        lookback_period: lookback period (month) / type: int
        inv_start: "YYYY-MM" / type: str
        inv_end: "YYYY-MM" / type: str
        amount: 투자금액 / type: int
        """
        self.data = data
        self.data_name = data_name
        self.ff_data = ff_data
        self.reopt_period = reopt_period
        self.rebal_period = rebal_period
        self.lookback_period = lookback_period
        self.inv_start = inv_start
        self.inv_end = inv_end
        self.amount = amount

        self.method_dict = {"Lasso":0, "Lasso_n":1, "Ridge":2, "Equal_weight":3, 
                            "Mean_variance":4, "Risk_parity":5, "Factor1":6, "Factor2":7, "Factor3":8, "AllSeason":9 ,"GB":10}

        
        # result(dataframe)
        self.res_df = pd.DataFrame([])
        self.res_df.index.name = "Date"
        
        self.weight_df_dict = dict()
        
    # date(YYYY-MM)형식을 year, mon 분리해서 int로 return
    # output: y(year,int), m(month, int)
    def _date_str_to_int(self, date):
        y, m = date.split('-')
        y, m = int(y), int(m)
        return y,m


    # date(YYYY-MM형식)와 mon(개월 수) 연산
    # output: 연산 후 date(YYYY-MM형식)
    def _cal_date_with_month(self, date, mon, oper='-'):
        y, m = self._date_str_to_int(date)
        if oper == '-':

            if m-mon < 1:
                m= m-mon
                while m < 1:
                  y -= 1
                  m = m + 12
            else:
                m -= mon

        elif oper == '+':
            if m+mon > 12:
                m=m+mon
                while m > 12:
                  y += 1
                  m = m - 12
            else:
                m += mon

        return '{0:04d}-{1:02d}'.format(y,m)

    # 최적화 문제를 풀기 위해 lookback기간 동안 필요한 estimate 계산
    def _df_to_estimate(self, df):
        """
            === Description ===

        함수 설명: 최적화 문제를 풀기 위해 필요한 estimate를 계산

        * df: estimate 계산에 이용

        output: estimate list 
                ex) return, variance, covariance matrix (추후에 list에 다 넣기에는 형식이 애매하면 dictionary를 이용하거나,
                                                        새로 class 작성해서 넘겨주는 방식도 고려해볼 수 있을수도)

          est_lst[0]: mean lst ex) [0.5,0.3, ..., 0.7] 
          est_lst[1]: var lst ex) [...]
          est_lst[2]: cov matrix ex) [[...],[...],...]
          est_lst[3]: holding-period return 


        """
        n=4
        est_lst = [0 for i in range(n)]

        est_lst[0] = df.mean()
        est_lst[1] = df.var()
        est_lst[2] = df.cov()

        #lookback 기간동안 발생한 holding-period return
        s = {}
        for i in df.columns:
            val = df.get(i)
            hldprd_ret = 1.0
            for j in val : 
                j += 1   #ror -> total return 곱연산으로 기간의 수익을 가져오기 위함.
                hldprd_ret*=j
                s.update({i:hldprd_ret})

        est_lst[3] = pd.Series(s)
        est_lst[3] = est_lst[3] - 1 #total return -> ror

        return est_lst
    
    # 자산 평균 수익률과 계산된 가중치를 이용한 포트폴리오 return 계산
    def _pfo_return(self, mean, weight_lst):
        """
        === Description ===
        포트폴리오 return 계산

        * parameter *
        #est_mean: 특정 기간동안 estimate한 자산들의 평균 수익률 /type:Series
        mean: 수익률 /type:Series
        weight_lst: (가정)최적화를 통해 구한 weight / type:Series

        output: pfo return / type: float

        """
        ptf_ret = mean @ weight_lst

        return ptf_ret
    
    # 포트폴리오 리밸런싱 진행
    def _rebalance_pfo(self, pfo_amount, weight_lst):
        """
        === Description ===
        포트폴리오 rebalancing

        * parameter *
        pfo_amount: 해당 달 수익률이 더해져 바뀐 산업군 별 porfolio amount
        weight_lst: 각 산업군에 할당할 가중치
        """
        
        total = sum(pfo_amount)
        res_rebal = total * weight_lst

        return res_rebal
    
    # 백테스트 진행
    def start_backtest(self, method, k=1, gamma=1, user_input = 10):
        # 입력된 하나의 method만을 이용해 백테스트가 진행되며
        # 백테스트 결과는 res_df에 모델별로 저장됨
        print("Method:", method)
        
        tmp_res = pd.DataFrame([], columns = ["{}_{}_return".format(self.data_name, method),"{}_{}_balance".format(self.data_name, method)])
        tmp_end = self._cal_date_with_month(self.inv_end, 1,'+')
        tmp_date = self.inv_start
        
        weight_df = pd.DataFrame([], columns = self.data.columns)
        weight_df.index.name = "Date"
        
        # weight_lst 초기화
        weight_dict = {}
        for i in self.data.columns:
            weight_dict.update({i:0.1})
        weight_lst = pd.Series(weight_dict)

        # rebalance chcker # (12/12) 사용자 입력 rebalance 기간으로 리벨런스하는 작업진행.
        # reopt_chk >= self.reopt_period 가 참 일때(if) -> re-optimize 진행, 진행후 reopt_chk = 1
        # reopt_chk >= self.reopt_period 가 거짓 일때(else) -> reopt_chk++       
        reopt_chk = 1 #
        opt_flag = True # 최초 투자시 weight 분배가 필요하므로 flag로 최초1회 optimize 함.
        
        # rebal_chk >= self.rebal_period 가 참 일때(if) -> 리벨런스 진행, 리벨런스 진행후 rebal_chk = 1
        # rebal_chk >= self.rebal_period 가 거짓 일때(else) -> rebal_chk++       
        rebal_chk = 1 # 최초투자시점 부터 리벨런스가 필요하지 않으므로 1로 시작함.

        # inital_amount 가정
        size = len(self.data.columns)
        pfo_amount = np.array([1.0/size]*size)*self.amount


        
        tmp_res.loc["Initial"] = [0, self.amount] #res_df 초기값

        # 백테스트
        while tmp_date != tmp_end:
            # re-optimize 주기 확인후 re-optimize
            if reopt_chk >= self.reopt_period or opt_flag :
                
                # lookback 기간을 scope 하는 작업
                look_start = self._cal_date_with_month(tmp_date, self.lookback_period,'-')
                lookback_df = self.data.loc[look_start:tmp_date][:-1]

                # lookback_df를 이용한 estimate 계산 함수
                est_lst = self._df_to_estimate(lookback_df) #est_lst <- mean(산술평균) , var, cov

                # 구한 estimate를 이용해 weight를 구하는 함수(최적화)
                pfo_opt = Portfolio_Optimization(est_lst, self.data.loc[tmp_date])

                sel_num = self.method_dict[method]
                if sel_num == 0: # Lasso
                    weight_lst = pfo_opt.Lasso(k)
                elif sel_num == 1: #Lasso_n
                    print(look_start, tmp_date)
                    weight_lst = pfo_opt.Lasso_n(user_input, len(self.data.columns))
                elif sel_num == 2: #Ridge
                    weight_lst = pfo_opt.Ridge(gamma)
                elif sel_num == 3: #EQW
                    weight_lst = pfo_opt.Equal_weight()
                elif sel_num == 4: #MV
                    weight_lst = pfo_opt.Mean_variance()
                elif sel_num == 5: #RP
                    weight_lst = pfo_opt.Risk_parity()
                elif sel_num == 6: #F1
                    weight_lst = pfo_opt.Factor1(lookback_df, self.ff_data.loc[look_start:tmp_date][:-1])
                elif sel_num == 7: #F2
                    weight_lst = pfo_opt.Factor2(lookback_df, self.ff_data.loc[look_start:tmp_date][:-1])
                elif sel_num == 8: #F3
                    weight_lst = pfo_opt.Factor3(lookback_df, self.ff_data.loc[look_start:tmp_date][:-1])
                elif sel_num == 9: #AS
                    weight_lst = pfo_opt.AllSeason()
                elif sel_num == 10: #GB
                    weight_lst = pfo_opt.GB()
                else:
                    print("ERROR 0: 잘못된 method 입력입니다.(Equal_weight, Mean_variance, Risk_parity, Ridge, Lasso, Lasso_n)")
                    return 0


                reopt_chk = 1

                if opt_flag == True :
                    reopt_chk = 2
                    opt_flag = False

            else :
                reopt_chk = reopt_chk + 1
          
            #각 시점 자산별 weight를 어떻게 나눴는지 확인하기 위한 DataFrame 저장
            weight_df.loc[tmp_date] = np.array(weight_lst)

            pfo_amount *= (self.data.loc[tmp_date] + 1) #각 산업군 (amount * (1+수익률))
            tmp_res.loc[tmp_date] = [self._pfo_return(self.data.loc[tmp_date], weight_lst), sum(pfo_amount)]

            # 리벨런스 주기확인후 리벨런싱
            if rebal_chk >= self.rebal_period :
                pfo_amount = self._rebalance_pfo(pfo_amount, weight_lst)
                rebal_chk = 1
            else :
                rebal_chk = rebal_chk + 1   


            # 날짜 계산
            tmp_date = self._cal_date_with_month(tmp_date, 1, '+')

        # 백테스트 결과 최종 res_df에 추가(모델별로 결과 저장됨)
        self.res_df = pd.concat([self.res_df, tmp_res], axis = 1)
    
        # 백테스트 결과 최종 weight 추가(모델별로 결과 저장됨)
        self.weight_df_dict[method] = weight_df
        
        return self.res_df

# 4. Performance Evaluation

In [ ]:
def add_market_df(backtest_df_lst, market, amount):
    
    """
    === Description ===
    여러 자산들의 백테스트 결과와 해당 기간동안 벤치마크 수익률 데이터프레임을 합쳐 하나의 데이터프레임으로 생성
    output은 성능평가(Performance Evaluation)와 시각화(Visualization)에 사용됨

    * parameter *
    backtest_df_lst: bactest후 나온 dataframe들을 담은 리스트 / type: list / ex) [df1, df2]
    market: benchmark dataframe / type:DataFrame
    amount: 투자금액 (backtest_df_lst 내 모든 초기투자금액과 동일해야 하며, 그렇지 않으면 에러 발생)
    """

    # ERROR 체크1: list 형식이 아닌 경우
    if not isinstance(backtest_df_lst, list):
        print("[ERROR]\n list형식으로 입력하세요.")
        return -1
    
    # ERROR 체크2: 데이터가 입력되지 않은 경우
    if not backtest_df_lst:
        print("[ERROR]\n backtest_df_lst에 데이터를 입력해주세요")
        return 0
    elif len(backtest_df_lst) > 1:
        # 1차 사이즈 확인
        size = len(backtest_df_lst[0])
        for i in range(1, len(backtest_df_lst)):
            if len(backtest_df_lst[i]) != size:
                print("[ERROR]\n 데이터의 총 투자기간이 다릅니다.")
                return 1
        
            if size != sum(backtest_df_lst[0].index == backtest_df_lst[1].index):
                print("[ERROR]\n 데이터의 세부 투자기간이 다릅니다.")
                return 2
    
    for i in range(len(backtest_df_lst)):
        col_name = ' '
        for col in backtest_df_lst[i]:
            if 'balance' in col:
                col_name = col
                break
        bt_amount = backtest_df_lst[i].loc['Initial',col_name]
        if amount != bt_amount:
            split_lst = col_name.split('_')
            print("[ERROR]\n {}자산의 초기 balance가 {}이므로 입력된 {}와 다릅니다.".format(split_lst[0], bt_amount, amount))
            return 3

    # 타입변경 (object -> float64)
    float_df_lst= []
    for i in range(len(backtest_df_lst)):
        float_df_lst.append(backtest_df_lst[i].astype('float64'))

    res = pd.concat(float_df_lst, axis = 1)
    
    # 벤치마크 수익률, 투자금액 추가
    res["Market_return"] = market[backtest_df_lst[0].index[1]:backtest_df_lst[0].index[-1]]
    res["Market_balance"] = amount

    for i in res.index[1:]:
        amount *= (res.Market_return.loc[i] + 1)
        res.loc[i,"Market_balance"] = amount
        
    res.loc['Initial','Market_return'] = 0
    
    return res

In [ ]:
class Performance_Evaluation:
    '''
    Description: 백테스트 결과로 모델별 성능 평가 및 벤치마크와의 비교
     (Constructor: param)
      - df: 벤치마크와 백테스트 결과 종합 데이터프레임
      - option: 데이터 수익률 기준(day,month year)
     (Constructor: default param)
      - size: 입력된 데이터프레임의 크기
      - compare_df: 성능평가 계산 결과
      
    Method
     (Public)
     * 모든 평가척도는 강의자료를 참고하여 구현하였습니다.
      - arithmetic_mean: arithmetic mean 계산
      - annualized_art: annualized arithmetic mean 계산
      - hold_period_ret: holding period return 계산
      - annualized_geo: annualized geometric mean 계산
      - variance: variance 계산
      - volatility: volatility 계산
      - annual_vol: annual volaility 계산
      - Sharpe_ratio: Sharpe ratio 계산
      - Sharpe_ratio_annual: annualized sharpe ratio 계산
      - covariance: Market과의 covariance 계산
      - Bata: Market과의 Beta 계산
      - Alpha: Market과의 Alpha 계산
      - VaR: Value at Risk 계산
      - CVaR: Conditional Value at Risk 계산
      - MDD: Max Drawdown 계산
      
      - compare_pfo_weigh_bm: 입력된 자산들의 백테스트 결과와 벤치마크의 평가척도를 계산해 비교
     (Private)
      - _error_check: 에러 확인
      - _clean_val: 계산 결과 값 반올림
     
    '''
    # Performance Evaluation class constructor
    def __init__(self, df, option='month'):
        self.df= df.iloc[1:]
        self.size = len(df)
        self.option = option
        self.compare_df = pd.DataFrame([])
        
        
    # 에러 확인
    def _error_check(self, method_lst, asset_name_lst):
        if not isinstance(method_lst,list) or not isinstance(asset_name_lst,list):
            print("[ERROR] list 형식으로 입력하세요")
            return False
        
        for i in asset_name_lst:
            for k in method_lst:
                for j in ["return","balance"]:
                    col_name= '{}_{}_{}'.format(i,k,j)
                    if col_name not in list(self.df.columns):
                        print("[ERROR] \'{}\'자산 백태스트 결과에 \'{}\' 모델의 결과가 없습니다.".format(i,k))
                        return False
        
        return True       

    # 결과 값 반올림(5)
    def _clean_val(self, num, is_clean):
        if is_clean:
            return round(num,5)
        else:
            return num
    
    # arithmetic mean 계산
    def arithmetic_mean(self, method, is_clean=True):
        col_name = "{}_return".format(method)
        mean = self.df[[col_name]].mean().item()
        
        return self._clean_val(mean, is_clean)
    
    # annualized arithmetic mean 계산
    def annualized_art(self, method, is_clean=True):


        tmp = 12 #month
        if self.option == 'week':
            tmp = 52
        elif self.option == 'day':
            tmp = 252
        
        mean = tmp * self.arithmetic_mean(method, False)
        
        return self._clean_val(mean, is_clean)
    
    # holding period return 계산
    def hold_period_ret(self, method, is_clean = True):
        col_name = "{}_balance".format(method)
        
        beg = self.df[[col_name]].iloc[0]
        end = self.df[[col_name]].iloc[-1]
        
        ret = (end/beg-1).item()

        return self._clean_val(ret,is_clean)
    
    # annualized geometric mean 계산
    def annualized_geo(self, method, is_clean=True):

        tmp = 12 #month
        if self.option == 'week':
            tmp = 52
        elif self.option == 'day':
            tmp = 252

        num_of_year = self.size/tmp
        h_period_ret = self.hold_period_ret(method, False)

        geo_num = (1+h_period_ret)**(1/num_of_year)-1
        
        return self._clean_val(geo_num, is_clean)
    
    # variance 계산
    def variance(self,method, is_clean=True):
        col_name = "{}_return".format(method)
        var = self.df[[col_name]].var().item()
        
        return self._clean_val(var, is_clean)
    
    # volatility 계산
    def volatility(self, method, is_clean=True):
        var = self.variance(method, False)
        vol = np.sqrt(var).item()
        
        return self._clean_val(vol, is_clean)
    
    # annual volaility 계산
    def annual_vol(self, method, is_clean=True):
        tmp = 12
        if self.option == 'week':
            tmp = 52
        elif self.option == 'day':
            tmp = 252

        vol = self.volatility(method, False)
        res = np.sqrt(tmp) * vol
        
        return self._clean_val(res,is_clean)

    # Sharpe ratio 계산
    def Sharpe_ratio(self, method, risk_free = 0.001, is_clean=True):
        mean = self.arithmetic_mean(method, False)
        vol = self.volatility(method, False)
        
        sr = (mean-risk_free)/vol
        return self._clean_val(sr, is_clean)
    
    # annualized sharpe ratio 계산
    def Sharpe_ratio_annual(self, method, risk_free = 0.001, is_clean=True):
        a_mean = self.annualized_art(method, False)
        a_vol = self.annual_vol(method, False)
        
        sra = (a_mean-12*risk_free)/a_vol
        return self._clean_val(sra,is_clean)
    
    # Market과의 covariance 계산
    def covariance(self, method, is_clean = True):
        if method == "Market":
            print("*ERROR: method는 Market이 될 수 없습니다.")
            return np.nan

        col_name= "{}_return".format(method)
        covar = self.df[[col_name, "Market_return"]].cov().iloc[0,1]

        return self._clean_val(covar, is_clean)
    
    # Market과의 Beta 계산
    def Beta(self, method, is_clean=True):
        if method == "Market":
            print("*ERROR: method는 Market이 될 수 없습니다.")
            return np.nan
        
        covar = self.covariance(method,False)
        var_market = self.variance("Market", False)

        beta = covar/var_market
        return self._clean_val(beta, is_clean)
    
    # Market과의 Alpha 계산
    def Alpha(self, method, risk_free = 0.001, is_clean=True):
        if method == "Market":
            print("*ERROR: method는 Market이 될 수 없습니다.")
            return np.nan        
        
        mean_pfo = self.arithmetic_mean(method, False)
        beta = self.Beta(method, False)

        mean_market = self.arithmetic_mean("Market", False)
        
        alpha = mean_pfo - risk_free - beta*(mean_market -risk_free)

        return self._clean_val(alpha, is_clean)
    
    # Value at Risk 계산
    def VaR(self, method, interval = 90, is_clean=True):
        
        col_name= "{}_return".format(method)
        sort_df = self.df[[col_name]].sort_values(col_name, ascending = True)

        VaR_90 = sort_df[col_name].quantile(0.1)
        VaR_95 = sort_df[col_name].quantile(0.05)
        VaR_99 = sort_df[col_name].quantile(0.01)
        
        if interval == 90:
            return self._clean_val(VaR_90, is_clean)
        elif interval == 95:
            return self._clean_val(VaR_95, is_clean)
        elif interval == 99:
            return self._clean_val(VaR_99, is_clean)
        else:
            return [self._clean_val(VaR_90, is_clean), 
                    self._clean_val(VaR_95, is_clean), 
                    self._clean_val(VaR_99, is_clean)]        
        
    # Conditional Value at Risk 계산
    def CVaR(self, method, interval = 90, is_clean=True):
        
        col_name= "{}_return".format(method)
        sort_df = self.df[[col_name]].sort_values(col_name, ascending = True)

        if interval in [90,95,99]:
            VaR_num = self.VaR(method, interval, False)
            CVaR_int = sort_df[sort_df[col_name] <= VaR_num].mean().item()
            return self._clean_val(CVaR_int, is_clean)
        else:
            VaR_lst = self.VaR(method,'all', False)
            tmp_cvar_lst = []
            for v in VaR_lst:
                tmp_cvar = sort_df[sort_df[col_name] <= v].mean().item()
                tmp_cvar_lst.append(self._clean_val(tmp_cvar, is_clean))
            return tmp_cvar_lst
        
    # Max Drawdown 계산
    def MDD(self, method, is_clean=True):
        col_name = '{}_balance'.format(method)
        m_bal_df = self.df[[col_name]].copy()
        m_bal_df["Highest_past"] = m_bal_df.cummax()
        m_bal_df["DD"] = m_bal_df[col_name]/m_bal_df["Highest_past"] - 1.0
        mdd = m_bal_df["DD"].min()

        return self._clean_val(mdd, is_clean)

    # 입력된 자산들의 백테스트 결과와 벤치마크의 평가척도를 계산해 비교
    def compare_pfo_with_bm(self, method_lst, asset_name_lst, risk_free=0.001, is_clean=True):
        
        if not self._error_check(method_lst, asset_name_lst):
            print("다시 입력하세요.")
            return -1
        
        # column 생성
        col_lst = []
        for asset_name in asset_name_lst:
            for method in method_lst:
                col_lst.append("{}_{}_pfo".format(asset_name, method))
        col_lst.append('Benchmark')
        
        compare_df = pd.DataFrame([], columns = col_lst)
        
        # 계산
        ari_mean_lst = []
        ari_mean_annual_lst = []
        vol_lst = []
        vol_annual_lst = []
        sr_lst = []
        var_lst = []
        cvar_lst = []
        mdd_lst = []
        market_flag = False
        
        # asset
        for asset_name in asset_name_lst:
            for method in method_lst:
                col_name = '{}_{}'.format(asset_name, method)
                
                ari_mean_lst.append(self.arithmetic_mean(col_name, is_clean))
                ari_mean_annual_lst.append(self.annualized_art(col_name, is_clean))
                vol_lst.append(self.volatility(col_name, is_clean))
                vol_annual_lst.append(self.annual_vol(col_name, is_clean))
                sr_lst.append(self.Sharpe_ratio_annual(col_name, risk_free, is_clean))
                var_lst.append(self.VaR(col_name,'all', is_clean))
                cvar_lst.append(self.CVaR(col_name,'all', is_clean))
                mdd_lst.append(self.MDD(col_name,is_clean))
          
        # market
        col_name = 'Market'
        ari_mean_lst.append(self.arithmetic_mean(col_name, is_clean))
        ari_mean_annual_lst.append(self.annualized_art(col_name, is_clean))
        vol_lst.append(self.volatility(col_name, is_clean))
        vol_annual_lst.append(self.annual_vol(col_name, is_clean))
        sr_lst.append(self.Sharpe_ratio_annual(col_name, risk_free, is_clean))
        var_lst.append(self.VaR(col_name,'all', is_clean))
        cvar_lst.append(self.CVaR(col_name,'all', is_clean))
        mdd_lst.append(self.MDD(col_name,is_clean))       
        
    
        compare_df.loc['Arithmetic mean(monthly)'] = ari_mean_lst
        compare_df.loc['Arithmetic mean(annualized)'] = ari_mean_annual_lst

        compare_df.loc["Volatility(monthly)"] = vol_lst
        compare_df.loc["Volatility(annualized)"] = vol_annual_lst

        compare_df.loc["Sharpe Ratio"] = sr_lst
        compare_df.loc["MDD"] = mdd_lst
        
        # Alpha, Beta
        a_lst = []
        b_lst = []
        for asset_name in asset_name_lst:
            for method in method_lst:
                col_name = '{}_{}'.format(asset_name,method)
                a_lst.append(self.Alpha(col_name, risk_free, is_clean))
                b_lst.append(self.Beta(col_name,is_clean))
            
        a_lst.append(np.nan)
        b_lst.append(np.nan)
        
        compare_df.loc["Alpha"] = a_lst
        compare_df.loc["Beta"] = b_lst
        
        # VaR, CVaR
        interval_lst = [90,95,99]
        for i in range(len(interval_lst)):
            tmp_var = []
            tmp_cvar = []
            for k in var_lst:
                tmp_var.append(k[i])
            for ck in cvar_lst:
                tmp_cvar.append(ck[i])
            
            compare_df.loc["VaR {}%".format(interval_lst[i])] = tmp_var
            compare_df.loc["CVaR {}%".format(interval_lst[i])] = tmp_cvar
        self.compare_df = compare_df
        return compare_df




# 5. Visualization

In [ ]:
class Visualization:
    '''
    Description: 백테스트 결과로 모델별 시각화 및 벤치마크와의 비교
     (Constructor: param)
      - df: 벤치마크와 백테스트 결과 종합 데이터프레임
      - method_lst: 비교하고픈 최적화 모델의 목록
      - asset_name_lst: 사용한 자산의 이름 목록
     (Constructor: default param)
      - df_dd: drawdown을 시각화하기 위해 사용되는 데이터프레임(Initial제거)
      - amount: balance 계산 시 초기 투자금액
      
    Method
     (Public)
      - viz_pfo_return_with_market: 백테스트 결과와 벤치마크의 수익률 변동 추이 그래프 시각화
      - viz_pfo_balance_with_market: 백테스트 결과와 벤치마크의 투자금액 변화 추이 그래프 시각화
      - viz_pfo_balance_with_market_log: 투자금액 변화 추이 로그 스케일 그래프 시각화
      - viz_pfo_dd_with_market: 백테스트 결과와 벤치마크의 drawdown 그래프
      - viz_all: 모든 그래프 화면에 출력
     (Private)
     - _error_check: 에러 확인
     
    '''
    
    # Visualization class constructor
    def __init__(self, df, method_lst, asset_name_lst):       
        self.df = df
        self.df_dd = df.iloc[1:]
        self.amount = df['Market_balance'].iloc[0]
        self.method_lst = method_lst
        self.asset_name_lst = asset_name_lst
        
        if not self._error_check():
            print("다시 입력하세요.")
        
    # 에러 확인
    def _error_check(self):
        if not isinstance(self.method_lst,list) or not isinstance(self.asset_name_lst,list):
            print("[ERROR] list 형식으로 입력하세요")
            return False
        
        for i in self.asset_name_lst:
            for k in self.method_lst:
                for j in ["return","balance"]:
                    col_name= '{}_{}_{}'.format(i,k,j)
                    if col_name not in list(self.df.columns):
                        print("[ERROR] \'{}\'자산 백태스트 결과에 \'{}\' 모델의 결과가 없습니다.".format(i,k))
                        return False
        
        return True       

    # 백테스트 결과와 벤치마크의 수익률 변동 추이 그래프 시각화
    def viz_pfo_return_with_market(self):
        if not self._error_check():
            print("다시 입력하세요.")
            return -1
        
        # benchmark
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.df.index, y=self.df.Market_return,
                            mode='lines',
                            name='Market return'))
        
        # asset
        for name in self.asset_name_lst:
            for i in self.method_lst:
                col_name = '{}_{}'.format(name,i)
                fig.add_trace(go.Scatter(x=self.df.index, y=self.df["{}_return".format(col_name)],
                                    mode='lines',
                                    name='{} {} pfo return'.format(name, i)))
            
        fig.update_layout(
            title="Portfolio growth with Market(return)",
        #     width=1000,
        #     height=400,
            xaxis = dict(title_text="date",),
            yaxis=dict(
                title_text="return",
            ),
            hovermode = 'x'
        )
        fig.add_shape(type='line',
                x0=0,
                y0=0,
                x1=len(self.df),
                y1=0,
                # opacity=0.5,
                line=dict(color='black',width=1)
        )
        fig.update_xaxes(type='category')
        fig.show()

    # 백테스트 결과와 벤치마크의 투자금액 변화 추이 그래프 시각화
    def viz_pfo_balance_with_market(self):
        if not self._error_check():
            print("다시 입력하세요.")
            return -1
        
        # benchmark
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=self.df.index, y=self.df.Market_balance,
                            mode='lines',
                            name='Market balance'))
        
        # asset
        for asset_name in self.asset_name_lst:
            for i in self.method_lst:
                col_name = '{}_{}'.format(asset_name, i)
                fig.add_trace(go.Scatter(x=self.df.index, y=self.df["{}_balance".format(col_name)],
                                    mode='lines',
                                    name='{} {} pfo balance'.format(asset_name, i)))

        fig.add_shape(type='line',
                x0=0,
                y0=self.amount,
                x1=len(self.df),
                y1=self.amount,
                # opacity=0.5,
                line=dict(color='black', width=1)
        )
        fig.update_layout(
            title="Portfolio growth with Market(balance)",
        #     width=1000,
        #     height=400,
            xaxis = dict(title_text="date",),
            yaxis=dict(
                title_text="balance",
            ),
            hovermode = 'x'
        )
        fig.update_xaxes(type='category')

        fig.show()
        
    # 투자금액 변화 추이 로그 스케일 그래프 시각화
    def viz_pfo_balance_with_market_log(self):
        if not self._error_check():
            print("다시 입력하세요.")
            return -1
        
        fig = go.Figure()
        
        # benchmark
        fig.add_trace(go.Scatter(x=self.df.index, y=self.df.Market_balance,
                            mode='lines',
                            name='Market balance'))
        # asset
        for asset_name in self.asset_name_lst:
            for i in self.method_lst:
                fig.add_trace(go.Scatter(x=self.df.index, y=self.df["{}_{}_balance".format(asset_name, i)],
                                    mode='lines',
                                    name='{} {} pfo balance'.format(asset_name, i)))
        fig.add_shape(type='line',
                x0=0,
                y0=self.amount,
                x1=len(self.df),
                y1=self.amount,
                # opacity=0.5,
                line=dict(color='black', width=1)
        )
        
        # logscale 변화
        fig.update_layout(
            title="Portfolio growth with Market(balance)-log sclae",
        #     width=1000,
        #     height=400,
            xaxis = dict(title_text="date",),
            yaxis=dict(
                title_text="balance(log)",
            ),
            hovermode = 'x',
            yaxis_type="log"
        )
        fig.update_xaxes(type='category')

        fig.show()


    # Description : Drawdown 그래프 그리기
    # input : none
    # output : none
    def viz_pfo_dd_with_market(self):
        
        if not self._error_check():
            print("다시 입력하세요.")
            return -1
        
        import warnings
        warnings.filterwarnings(action = 'ignore')      
        fig = go.Figure()
        window = 1

        # benchmark의 drawdown 계산
        Roll_Max = self.df_dd['Market_balance'].cummax()
        Daily_Drawdown = self.df_dd['Market_balance']/Roll_Max - 1.0
        self.df_dd['Market_DD'] = Daily_Drawdown

        # benchmark의 그래프 plot
        fig.add_trace(go.Scatter(x=self.df_dd.index, y=self.df_dd.Market_DD,
                          mode='lines',
                          name='Market return'))
        for asset_name in self.asset_name_lst:
            for i in self.method_lst:
                # 자산군들의 drawdown 계산
                Roll_Max = self.df_dd['{}_{}_balance'.format(asset_name,i)].cummax()
                Daily_Drawdown = self.df_dd['{}_{}_balance'.format(asset_name,i)]/Roll_Max - 1.0
                self.df_dd['{}_{}_DD'.format(asset_name,i)] = Daily_Drawdown
                
                # 자산군들의 그래프 plot
                fig.add_trace(go.Scatter(x=self.df_dd.index, y=self.df_dd["{}_{}_DD".format(asset_name,i)],
                                  mode='lines',
                                  name='{} {} pfo return'.format(asset_name,i)))
        # plot의 layout 설정
        fig.update_layout(
          title="Portfolio drawdown with Market(return)",
        #     width=1000,
        #     height=400,
          xaxis = dict(title_text="date",),
          yaxis=dict(
              title_text="return",
          ),
          hovermode = 'x'
        )

        fig.show()
        warnings.filterwarnings(action = 'default')
        
    # 모든 그래프 화면에 출력
    def viz_all(self):
        if not self._error_check():
            print("다시 입력하세요.")
            return -1
        self.viz_pfo_return_with_market()
        self.viz_pfo_balance_with_market()
        self.viz_pfo_balance_with_market_log()
        self.viz_pfo_dd_with_market()

# 6. Simulation

In [ ]:
class Simulation_MC:
    '''
    Description: Monte-Carlo simulation을 통한 분석
     (Constructor: default param)
      - asset_option: 자산 데이터 수익률 기준(month) *현재 시뮬레이션은 month 기준으로만 작동합니다.
      - pfo_mean: portfolio 평균 수익률
      - pfo_vol: portfolio volatility
      - simul_df: simulation 결과 저장 데이터프레임
      - fin_price: simulation 최종 금액 저장 데이터프레임
      - quan_table: 분위별 기준 값 저장
      - out_except_df: outlier를 제거한 simulation 최종 금액 저장 데이터프레임
      - MDD: 각 시뮬레이션 동안의 Max drawdown 값 저장 데이터프레임
      - investment: 투자금액
      
    Method
     (Public)
      - set_portfolio: 시뮬레이션에 이용할 자산 및 가중치 설정(자동/수동)
      - start_simulation: 시뮬레이션 진행
      - quantile_table: 분위별 기준 값 계산
      - viz_box_plot: quartile box plot 시각화
      - remove_outlier: outlier 제거
      - make_MDD: Max drawdown 계산
      - viz_graph: End balance frequency histogram과 MDD 그래프 화면에 출력
    '''
    
    # Simulation_MC class constructor
    def __init__(self):
        self.asset_option = 0
        self.pfo_mean = 0
        self.pfo_vol = 0
        self.simul_df = pd.DataFrame()
        self.fin_price = pd.DataFrame()
        self.quan_table = pd.DataFrame([], columns = ["Quantile", "Price"])
        self.out_except_df = pd.DataFrame()
        self.MDD = pd.DataFrame()
        self.investment=0

    # 시뮬레이션에 이용할 자산 및 가중치 설정(자동/수동)
    def set_portfolio(self, is_auto = True, asset_option = 'month', metric = 'Mean_variance', 
                  asset_select = 10, asset_ret = [], weight_lst = []):
        '''
        is_auto = True(default) -> 입력된 metric에 따라 자동으로 Portfolio Optimization을 통해 나온 가중치를 이용
                                   * metric
                                   * asset_select = 10(default) -> 사용할 특정 자산군 선택 가능
                  False         -> 사용자가 입력한 asset return과 weight lst를 이용해 포트폴리오 구성
                                   * asset_ret, weight_lst
        asset_option = 'month'(default) -> asset의 return 기간 기준을 함께 입력
        '''
        # 가중치 자동 계산
        if is_auto:
            if len(asset_ret) == 0:
                if asset_option != 'month':
                    print("[ERROR -1]: 자산 기간 설정이 잘못되었습니다.(default asset 사용시 month 필수)")
                    return -1
                
                # asset_select대로(asset option 건드렸나 확인)
                else:
                    try:
                        tmp =  web.DataReader('{}_Industry_Portfolios'.format(asset_select), 'famafrench', start='1990-01-01', end='2020-09-01')
                    except:
                        print("[ERROR -1]: asset_select의 설정이 잘못되었습니다.(5,10,12,17,30,38,48,49 중 선택 가능)")
                        return -1
                    
                    asset_ret = tmp[0]/100
#             else:
#                 ret = asset_ret.copy()
            
            est_lst = [asset_ret.mean(), asset_ret.var(), asset_ret.cov()]
            
            self.asset_option = asset_option

            # Portfolio_Optimization 만들어서
            pfo_opt = Portfolio_Optimization(est_lst, asset_ret)
            if metric == 'Lasso':
                weight_lst = pfo_opt.Lasso()
            elif metric == 'Lasso_n':
                weight_lst = pfo_opt.Lasso_n()
            elif metric == 'Ridge':
                weight_lst = pfo_opt.Ridge()
            elif metric == 'Equal_weight': #EQW
                weight_lst = pfo_opt.Equal_weight()
            elif metric == 'Mean_variance': #MV
                weight_lst = pfo_opt.Mean_variance()
            elif metric == 'Risk_parity': #RP
                weight_lst = pfo_opt.Risk_parity()
            else:
                print("ERROR -1: 해당 최적화 모델을 존재하지 않습니다.")
                return -1
            
            self.pfo_mean = asset_ret.mean() @ weight_lst
            self.pfo_vol = np.sqrt(weight_lst.T.dot(asset_ret.cov()).dot(weight_lst))
        
        # 가중치 수동 입력
        else:
            # 예외처리
            if len(weight_lst) == 0:
                print("[ERROR 0]: weight_lst의 입력이 없습니다.")
                return 0
            elif len(asset_ret) == 0:
                if asset_select != len(weight_lst):
                    print("[ERROR 0]: weight_lst의 개수가 default자산의 개수(10)와 일치하지 않습니다.")
                    return 0
                
                if asset_option != 'month':
                    print("[ERROR 0]: 자산 기간 설정이 잘못되었습니다.(default asset 사용시 month 필수)")
                    return 0
                tmp =  web.DataReader('{}_Industry_Portfolios'.format(asset_select), 'famafrench', start='1990-01-01', end='2020-09-01')
                asset_ret = tmp[0]/100 
            else:
                if len(weight_lst) != len(asset_ret.columns):
                    print("[ERROR 0]: 입력한 asset의 개수와 weight의 개수가 일치하지 않습니다.")
                    return 0
            
            # 포폴 mean, vol 구하기
            if asset_option in ['day', 'month', 'year']:
                self.asset_option = asset_option
                self.pfo_mean = asset_ret.mean() @ weight_lst
                self.pfo_vol = np.sqrt(weight_lst.T.dot(asset_ret.cov()).dot(weight_lst))
            else:
                print("[ERROR 1]: 잘못된 option입니다.")
                return 1
    
    # 시뮬레이션 진행
    def start_simulation(self, investment, duration,  
                         simul_num = 10000, sip = 0, v=10, option = 'month'):
        self.investment = investment
        self.simul_df = pd.DataFrame()
        self.fin_price = pd.DataFrame()
        rv = stats.t(v, self.pfo_mean, self.pfo_vol)
    
        for i in range(simul_num):
            price = investment
            
            tmp_price_lst = []
            for year in range(duration):
                new_ret = rv.rvs()
                price = price*(1+new_ret) + sip
                tmp_price_lst.append(price)
            self.simul_df[i] = tmp_price_lst
        
        self.fin_price = self.simul_df.iloc[-1,:].to_frame('Final_Price') #최종 값
        return self.fin_price.describe()
        
    # 분위별 기준 값 계산
    def quantile_table(self, ran = [0.1, 0.25, 0.5, 0.75, 0.9]):
        self.quan_table = pd.DataFrame([], columns = ["Quantile", "Price"])

        for i in range(len(ran)):
            self.quan_table.loc[i] = ['{}%'.format(round(ran[i]*100,1)), fin_price.quantile(ran[i]).item()]
        return self.quan_table
    
    # quartile box plot 시각화
    def viz_box_plot(self):
        if len(self.fin_price) == 0:
            print("'start_simulation'함수를 먼저 실행시켜주세요.")
            return 0
        fig = go.Figure()
        
        fig.add_trace(go.Box(x=self.fin_price['Final_Price']))

        fig.update_layout(title='Quartile',
                   xaxis_title='Month',
                   yaxis_title='Temperature (degrees F)')
        fig.show()
    
     # 모든 시뮬레이션 금액 변동 추이 시각화 그래프(시뮬레이션 횟수가 많으면 시각화할 때 시간이 오래걸립니다)
#     def viz_all_simulation(self):
#         if len(self.simul_df)==0:
#             print("'start_simulation'함수를 먼저 실행시켜주세요.")
#             return 0
    
#         fig = go.Figure()
#         for i in list(self.simul_df.columns):
#             fig.add_trace(go.Scatter(x=self.simul_df.index, y=self.simul_df[i],
#                                 mode='lines',
#                                 name=i))
#         fig.add_shape(type='line',
#                 x0=0,
#                 y0=self.investment,
#                 x1=max(list(self.simul_df.index)),
#                 y1=self.investment,
#                 line=dict(color='black',),
#                 xref='x',
#                 yref='y'
#                 )
#         fig.show()

    # outlier 제거
    def remove_outlier(self):
        if len(self.fin_price) == 0:
            print("'start_simulation'함수를 먼저 실행시켜주세요.")
            return 0
        
        self.out_except_df = pd.DataFrame()
        
        IQR = self.fin_price.quantile(0.75) - self.fin_price.quantile(0.25)
        out_IQR = IQR * 1.5
        lowest = (self.fin_price.quantile(0.25) - out_IQR).item()
        highest = (self.fin_price.quantile(0.75) + out_IQR).item()
        self.out_except_df = self.fin_price[(self.fin_price["Final_Price"] >= lowest) & (self.fin_price["Final_Price"] <= highest)]
        
        return self.out_except_df
    
    # Max drawdown 계산
    def make_MDD(self):
        out_simul_df = self.simul_df[list(self.out_except_df.index)]
        out_simul_high = out_simul_df.cummax()
        mdd_lst = []

        for i in list(out_simul_df.columns):
            mdd_lst.append((out_simul_df[i]/out_simul_high[i] - 1.0).min())
        
        self.MDD = pd.DataFrame(mdd_lst, columns=['MDD'])
    
    # End balance frequency histogram과 MDD 그래프 화면에 출력
    def viz_graph(self):
        if len(self.out_except_df)==0:
            self.remove_outlier()
            
        if len(self.MDD)==0:
            self.make_MDD()

        percent = round(len(self.out_except_df)/len(self.fin_price) *100,2)
        fig = make_subplots(rows=2, cols=1, subplot_titles=('Portfolio End Balance Histogram ({}%)'.format(percent), 
                                                            'Maximum Drawdown Histogram ({}%)'.format(percent)))
        # End balance Frequency histogram
        fig.append_trace(go.Histogram(x=self.out_except_df['Final_Price'],
            name='Price', 
            marker_color='#EB89B5',
            opacity=0.75), row=1,col=1)

        # Max Drawdown
        fig.append_trace(go.Histogram(x=self.MDD['MDD'],
            name='MDD', 
            marker_color='#9966CC',
            opacity=0.75), row=2, col=1)

        fig.update_xaxes(title_text="End Balance($)", row=1, col=1)
        fig.update_xaxes(title_text="MAX Drawdown", row=2, col=1)

        fig.update_yaxes(title_text="Frequency", row=1, col=1)
        fig.update_yaxes(title_text="Frequency", row=2, col=1)

        fig.update_layout(height=800, width=980,
                         bargap=0.01,
                         bargroupgap=0.1)

        fig.layout.xaxis2.tickformat=',.2%'

        fig.show()
        
